In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [11]:
data = pd.read_csv('C:\\Users\\Almas\\Downloads/Android_Malware.csv')
data = data.astype(str)
data.info()

C:\Users\Almas\AppData\Local\Temp\ipykernel_13348\1273077433.py:1: DtypeWarning: Columns (56,58,63) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('C:\\Users\\Almas\\Downloads/Android_Malware.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355630 entries, 0 to 355629
Data columns (total 86 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   Unnamed: 0                    355630 non-null  object
 1   Flow ID                       355630 non-null  object
 2    Source IP                    355630 non-null  object
 3    Source Port                  355630 non-null  object
 4    Destination IP               355630 non-null  object
 5    Destination Port             355630 non-null  object
 6    Protocol                     355630 non-null  object
 7    Timestamp                    355630 non-null  object
 8    Flow Duration                355630 non-null  object
 9    Total Fwd Packets            355630 non-null  object
 10   Total Backward Packets       355630 non-null  object
 11  Total Length of Fwd Packets   355630 non-null  object
 12   Total Length of Bwd Packets  355630 non-null  object
 13 

In [3]:
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(data['Label'])
for column in data.columns:
    data[column] = label_encoder.fit_transform(data[column])

In [4]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

In [5]:
X = torch.tensor(scaled_data[:, :-1], dtype=torch.float32)
y = torch.tensor(labels, dtype=torch.long)

In [6]:
class FCNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FCNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = torch.relu(self.fc1(x))
        out = self.fc2(out)
        return out

In [7]:
input_size = scaled_data.shape[1] - 1
hidden_size = 64
output_size = len(np.unique(labels))  

In [8]:
model = FCNModel(input_size, hidden_size, output_size)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:
epochs = 175
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}')

with torch.no_grad():
    outputs = model(X)
    _, predicted_labels = torch.max(outputs, dim=1)
    accuracy = accuracy_score(labels, predicted_labels.numpy())
    precision = precision_score(labels, predicted_labels.numpy(), average='weighted')
    recall = recall_score(labels, predicted_labels.numpy(), average='weighted')
    f1 = f1_score(labels, predicted_labels.numpy(), average='weighted')
    conf_matrix = confusion_matrix(labels, predicted_labels.numpy())
    print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}')
#     print('Confusion Matrix:')
#     print(conf_matrix)

Epoch [1/175], Loss: 1.4182475805282593
Epoch [2/175], Loss: 1.403229832649231
Epoch [3/175], Loss: 1.389157772064209
Epoch [4/175], Loss: 1.3759453296661377
Epoch [5/175], Loss: 1.3635214567184448
Epoch [6/175], Loss: 1.3518198728561401
Epoch [7/175], Loss: 1.3407930135726929
Epoch [8/175], Loss: 1.3303970098495483
Epoch [9/175], Loss: 1.3205922842025757
Epoch [10/175], Loss: 1.311354637145996
Epoch [11/175], Loss: 1.3026667833328247
Epoch [12/175], Loss: 1.294509768486023
Epoch [13/175], Loss: 1.286872386932373
Epoch [14/175], Loss: 1.279738187789917
Epoch [15/175], Loss: 1.27308988571167
Epoch [16/175], Loss: 1.266899585723877
Epoch [17/175], Loss: 1.2611372470855713
Epoch [18/175], Loss: 1.2557755708694458
Epoch [19/175], Loss: 1.2507835626602173
Epoch [20/175], Loss: 1.2461259365081787
Epoch [21/175], Loss: 1.2417813539505005
Epoch [22/175], Loss: 1.2377346754074097
Epoch [23/175], Loss: 1.2339696884155273
Epoch [24/175], Loss: 1.2304767370224
Epoch [25/175], Loss: 1.2272373437881

D:\programs\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.7795, Precision: 0.7403, Recall: 0.7795, F1-score: 0.7288
Confusion Matrix:
[[146300     11   1132      0]
 [ 23887  22411  21099      0]
 [  8584      3 108495      0]
 [ 14102   3290   6316      0]]
